Copyright (c) Microsoft Corporation. All rights reserved. 

Licensed under the MIT License.

# Quickstart: Image Classification using Automated ML

In this quickstart, you use automated machine learning in Azure Machine Learning service to  train an image classification model using the [MNIST](https://azure.microsoft.com/services/open-datasets/catalog/mnist/) dataset. This process accepts training data and configuration settings, and automatically iterates through combinations of different feature normalization/standardization methods, models, and hyperparameter settings to arrive at the best model.

You will learn how to:

> * Download a dataset and look at the data
> * Train a machine learning image classification model using autoML 
> * Explore the results


### Connect to your workspace and create an experiment

You start with importing some libraries and creating an experiment to track the runs in your workspace. A workspace can have multiple experiments, and all the users that have access to the workspace can collaborate on them. 

In [ ]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

In [ ]:
ws = Workspace.from_config()

# choose a name for your experiment
experiment_name = "ccfraud-automl-sdk"

experiment = Experiment(ws, experiment_name)

## Import Data

Before you train a model, you need to understand the data that you are using to train it. In this section you will:

* Download the MNIST dataset
* Display some sample images

### Download the MNIST dataset

Use Azure Open Datasets to get the raw MNIST data files. [Azure Open Datasets](https://docs.microsoft.com/azure/open-datasets/overview-what-are-open-datasets) are curated public datasets that you can use to add scenario-specific features to machine learning solutions for more accurate models. Each dataset has a corrseponding class, `MNIST` in this case, to retrieve the data in different ways.

Follow this [how-to](https://aka.ms/azureml/howto/createdatasets) if you want to learn more about Datasets and how to use them.




In [ ]:
# swithced dataset from mnist to credit card fraud

data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/creditcard.csv"
dataset = Dataset.Tabular.from_delimited_files(data)
training_data, validation_data = dataset.random_split(percentage=0.8, seed=223)
label_column_name = 'Class'


## Train



When you use automated machine learning in Azure ML, you input training data and configuration settings, and the process automatically iterates through combinations of different feature normalization/standardization methods, models, and hyperparameter settings to arrive at the best model. 
Learn more about how you configure automated ML [here](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train).


Instantiate an [AutoMLConfig](https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py) object. This defines the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize. 
|**enable_early_stopping**  | Stop the run if the metric score is not showing improvement.|
|**n_cross_validations**|Number of cross validation splits.|
|**training_data**|Input dataset, containing both features and label column.|
|**label_column_name**|The name of the label column.|

You can find more information about primary metrics [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train#primary-metric)

In [ ]:
automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": 'average_precision_score_weighted',
    "experiment_timeout_hours": 0.25, # This is a time limit for testing purposes, remove it for real use cases, this will drastically limit ability to find the best model possible
    "verbosity": logging.INFO,
    "enable_stack_ensemble": False
}

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             training_data = training_data,
                             label_column_name = label_column_name,
                             **automl_settings
                            )


Call the `submit` method on the experiment object and pass the run configuration. 

**Note: Depending on the data and the number of iterations an AutoML run can take a while to complete.**

In this example, we specify `show_output = True` to print currently running iterations to the console. It is also possible to navigate to the experiment through the **Experiment** activity tab in the left menu, and monitor the run status from there.

In [ ]:
local_run = experiment.submit(automl_config, show_output=True)

In [ ]:
local_run

### Analyze results

Below we select the best model from our iterations. The `get_output` method on `automl_classifier` returns the best run and the model for the run.

In [ ]:
best_run, best_model = local_run.get_output()
best_model


Now that the model is trained, we'll run the test data through the trained model to get the predicted values.

In [ ]:
# convert the test data to dataframe
X_test_df = validation_data.drop_columns(
    columns=[label_column_name]
).to_pandas_dataframe()
y_test_df = validation_data.keep_columns(
    columns=[label_column_name], validate=True
).to_pandas_dataframe()

In [ ]:
# call the predict functions on the model
y_pred = best_model.predict(X_test_df)
y_pred

### Calculate metrics for the prediction

Now visualize the data to show what our truth (actual) values are compared to the predicted values 
from the trained model that was returned.

## Control cost and further exploration

If you want to control cost you can stop the compute instance this notebook is running on by clicking the "Stop compute" button next to the status dropdown in the menu above.


If you want to run more notebook samples, you can click on **Sample Notebooks** next to the **Files** view and explore the notebooks made available for you there.